# Fast simulation perfromance parameterization 
Expert  configurable event display for fast simulzation studies  


## Contents 
[Import modules](#import) <br>
[Load trees, set aliases](#load) <br>
[XYZ display pt](#XYZdisplayPt) <br>
[XYZ display pt](#XYZdisplaytrack) <br>

In [1]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter, RandomForest
import plotly.express as px
import time
import pickle
from IPython.display import Image, clear_output
from RootInteractive.InteractiveDrawing.bokeh.CDSCompress import *
#
from fastSimulation import *

Welcome to JupyROOT 6.20/08
x bokehVisJS3DGraph.ts
x HistogramCDS.ts
x Histo2dCDS.ts
Import  CDSCompress.ts


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## 1.) Load trees and define derived variables:
<a id='load'></a>

* Covaraince matrix of Alice kalman track - stored as an array 
* ```(y,z,sing*phi),tgl(theta),qPt) -> (0,2,5,9,14)```
 * 0   
 * 1 2
 * 3 4 5 
 * 6 7 8 9
 * 10 11 12 13 14 

In [2]:
inputData = "/home2/miranov/github/fastMCKalman/data/alice3/fastParticle.list";
inputData = "${fastMCKalman}/data/alice3/fastParticle.list";
inputData = "/home2/miranov/github/fastMCKalman/data/testTPC/fastParticle.list";
#inputData = "/home2/miranov/github/fastMCKalman/data/run2/fastParticle.list";

In [3]:
ROOT.initTreeFast(inputData)
tree=ROOT.treeFast

Number of entries	1
0	0	/home2/miranov/github/fastMCKalman/data/testTPC/fastParticle.root
Number of entries	1
0	0	/home2/miranov/github/fastMCKalman/data/testTPC/fastParticle.root
Number of entries	1
0	0	/home2/miranov/github/fastMCKalman/data/testTPC/fastParticle.root


Info in <AliDrawStyle::ApplyStyle>: figTemplate


In [8]:
tree.SetMarkerStyle(25)
tree.SetMarkerSize(0.5)
ROOT.setAliases(tree)
tree.SetAlias("qPt","part.fParamMC[0].fP[4]")
tree.SetAlias("ptMC","part.fParamMC[0].fData.Pt()")
tree.SetAlias("ptIn","part.fParamIn[1].fData.Pt()")
tree.SetAlias("ptMCP","part.fParamMC[].fData.Pt()")
tree.SetAlias("tglMC","part.fParamMC[0].fP[3]")
tree.SetAlias("tglIn","part.fParamIn[1].fP[3]")
#
tree.SetAlias("sigmaY0","sqrt(part.fParamIn[1].fC[0])")
tree.SetAlias("sigmaZ0","sqrt(part.fParamIn[1].fC[2])")
tree.SetAlias("sigmaqPt0","sqrt(part.fParamIn[1].fC[14])")
#
tree.SetAlias("sigmaY0Rot","sqrt(part.fParamInRot[1].fC[0])")
tree.SetAlias("sigmaZ0Rot","sqrt(part.fParamInRot[1].fC[2])")
tree.SetAlias("sigmaqPt0Rot","sqrt(part.fParamInRot[1].fC[14])")
#
tree.SetAlias("gzMC","(part.fParamMC[].fP[1])")
tree.SetAlias("gzIn","(part.fParamIn[].fP[1])")
tree.SetAlias("rMC","sqrt(gxMC**2+gyMC**2)")
tree.SetAlias("snpMC","(part.fParamMC[].fP[2])")
tree.SetAlias("pIndex","Iteration$")
tree.SetAlias("trackID","i")
tree.SetAlias("snpMax","abs(part.fParamMC[fMaxLayer].fP[2])")
#
tree.SetAlias("meanptMC","part.getMean(0+0,0)");
tree.SetAlias("meanptMCInv","part.getMean(1+0,0)");
tree.SetAlias("meandEdxExp","part.getMean(2+0,0)");
tree.SetAlias("meandEdxExpInv","part.getMean(3+0,0)");
tree.SetAlias("X0Norm","geom.fLayerX0.fData[0]/0.0001");
tree.SetAlias("sigmaRPhi","geom.fLayerResolRPhi.fData[0]");
tree.SetAlias("sigmaPtRel0","sigmaqPt0*ptMC");
tree.SetAlias("sigmaPtRel0Mean","sigmaqPt0/meanptMCInv");
#
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5")


In [103]:
%%time
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5")
defaultCutTrack="fitCut"
kineParam=["sigmaPtRel0","qPt","ptMC",
                           "sigmaY0","sigmaZ0","sigmaqPt0",
                           "sigmaY0Rot","sigmaZ0Rot","sigmaqPt0Rot",
                           "pidCode","isSecondary","fMaxLayer","dEdxExp",
                          "snpMax","fMassMC"]
resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi"]
dfRes=tree2Panda(tree,kineParam+resolParam,
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=50000)


CPU times: user 37.8 s, sys: 248 ms, total: 38.1 s
Wall time: 38.2 s


In [40]:
dfRes[variableX].head(5)

,meanptMCInv,meandEdxExpInv,X0Norm,tglMC,Larm,sigmaRPhi
0,0.069228,0.631929,0.783510,-1.115627,247.011993,0.021562
1,0.172006,0.864513,0.783510,-1.025128,94.624001,0.174096
2,7.357378,0.184189,1.091051,-0.522104,71.716003,0.100000
3,0.046186,0.785471,0.783510,0.127566,232.071991,0.058487
4,0.033528,0.823039,0.783510,1.245421,247.011993,0.079817


In [104]:
%%time
dfResML=dfRes
npoints=dfRes.shape[0]
varFit='sigmaPtRel0'
variableX=resolParam
dataContainer = DataContainer(dfResML, variableX, varFit, [npoints//2,npoints//2])
fitter = Fitter(dataContainer)
fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=200, max_depth=12)
fitter.Register_Method('RF_10','RandomForest', 'Regressor', n_estimators=200, max_depth=10)

fitter.Fit()
fitter.printImportance()
#
for method in ['RF' , 'RF_10']:    
    dfResML= fitter.AppendOtherPandas(method,dfResML)
fitter.AppendStatPandas("RF",dfResML)
dfResML=fitter.AppendStatPandas("RF_10",dfResML)


RF
Larm 0.43077566586347465
meanptMCInv 0.29298779597020297
sigmaRPhi 0.1470050109448745
meandEdxExpInv 0.0940238282463073
X0Norm 0.020829687596440338
tglMC 0.01437801137870039
RF_10
Larm 0.4385485111786987
meanptMCInv 0.29615003902173775
sigmaRPhi 0.14962782810373793
meandEdxExpInv 0.08835580751816856
X0Norm 0.017318090660414228
tglMC 0.009999723517242815
CPU times: user 6.35 s, sys: 16 ms, total: 6.37 s
Wall time: 6.4 s


In [ ]:
dfResMLDraw["FitRatio"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")



In [105]:
dfResML.columns

Index(['sigmaPtRel0', 'qPt', 'ptMC', 'sigmaY0', 'sigmaZ0', 'sigmaqPt0',
       'sigmaY0Rot', 'sigmaZ0Rot', 'sigmaqPt0Rot', 'pidCode', 'isSecondary',
       'fMaxLayer', 'dEdxExp', 'snpMax', 'fMassMC', 'meanptMCInv',
       'meandEdxExpInv', 'X0Norm', 'tglMC', 'Larm', 'sigmaRPhi', 'RF', 'RF_10',
       'RFMean', 'RFMedian', 'RFRMS', 'RFMeanMean', 'RFMeanMedian',
       'RFMedianMedian', 'RFMeanRMS', 'RFMedianRMS', 'RFRMSMean',
       'RFRMSMedian', 'RF_10Mean', 'RF_10Median', 'RF_10RMS', 'RF_10MeanMean',
       'RF_10MeanMedian', 'RF_10MedianMedian', 'RF_10MeanRMS',
       'RF_10MedianRMS', 'RF_10RMSMean', 'RF_10RMSMedian'],
      dtype='object')

## Resolution plot - outliers in covariance matrix could be too small
* resolution at layer 0 as obtained by covariance matrix 
* problem similar in 2 rotation schema
  * rotate to local after each propagation
  * only one rotation

In [115]:
output_file("fastSimulDisplayResolution.html")
dfResMLDraw=dfResML
dfResMLDraw["FitRatio"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*0.01*(sigmaRPhi+0.0001)*sqrt(720./(Larm+4))/(0.3*0.5*(Larm/100.)**(2.)))")

dfResMLDraw["FitRatioHPT"]=dfResMLDraw.eval("log(sigmaPtRel0/ptResolHPT)")

histoArray = [
    {"name": "histoqPt", "variables": ["qPt"], "nbins": 50},
    {"name": "histoFitRatio", "variables": ["FitRatio"], "nbins": 50},
    {"name": "histoFitRatioHPT", "variables": ["FitRatioHPT"], "nbins": 50}

]

figureArray = [
    [['qPt'], ['sigmaPtRel0'], {"colorZvar":"pidCode"}],
    [['qPt'], ['RF'], {"colorZvar":"pidCode"}],
    [['qPt'], ['ptResolHPT'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatio'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatioHPT'], {"colorZvar":"pidCode"}],
    #
    [["qPt"],["histoqPt"],{}],
    [["FitRatio"],["histoFitRatio"],{}],
    [["FitRatioHPT"],["histoFitRatioHPT"],{}],
    ["tableHisto", {"rowwise": False}],
    ['table'],
    {"size": 5}
]

widgetParams=[
    #infironment
    ['range', ['dEdxExp']],
    ['range', ['meandEdxExpInv']],
    ['range', ['X0Norm']],
    ['range', ['pidCode']],
    ['range', ['sigmaRPhi']],
    #
    ['range', ['ptMC']],
    ['range', ['qPt']],
    ['range', ['tglMC']],
    ['range', ['Larm']],
    #
    ["range", ['FitRatio']],
    ["range", ['FitRatioHPT']],
    ["range", ['ptResolHPT']]
   
    #dummy
    
]
tooltips = [("qPt","@qPt"),("sigmaRPhi","@sigmaRPhi")]
widgetLayoutDesc=[ [0,1,2,3,4],[5,6,7,8],[9,10], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1,2, {'plot_height':170}],
    [3,4, {'plot_height':120}],
    [5,6,7, {'plot_height':120}],
    [8, {'plot_height':50}],
    {'plot_height':400,'sizing_mode':'scale_width'}
]

output_file("fastSimulDisplayResolution.html")
figC=bokehDrawSA.fromArray(dfResMLDraw, "fMaxLayer>4", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=20000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8,histogramArray=histoArray,)

compressCDSPipe
Compress 1 qPt .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 104057 267900 0.3884173198954834 1 qPt
Compress 2 sigmaPtRel0 .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 49625 267908 0.18523149737969752 2 sigmaPtRel0
Compress 3 RF .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 47437 267899 0.1770704631222961 3 RF
Compress 4 ptResolHPT .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 77783 267907 0.2903358254916818 4 ptResolHPT
Compress 5 FitRatio .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 107199 267905 0.40013810865791977 5 FitRatio
Compress 6 FitRatioHPT .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 93425 267908 0.34872045627603504 6 FitRatioHPT
Compress 7 dEdxExp .* [('relative', 8), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 47257 267904 0.17639527591973245 7 dEdxExp
Compre

In [118]:
dfResMLDraw["pMC"]=dfResMLDraw.eval("sqrt(1+tglMC**2)*ptMC")
dfResMLDraw["Beta"]=dfResMLDraw.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
dfResMLDraw["ptResolLPT"]=dfResMLDraw.eval("(0.2*0.5/Beta)*sqrt(Larm/100.)")

In [99]:
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*(sigmaRPhi+0.01)*sqrt(720./(Larm+4.))/(0.3*0.5*(Larm/100.)**2.))")

In [34]:
#Fast prototyping part

In [80]:

dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*sigmaRPhi*sqrt(720.)/(0.5*Larm**(2.5)))")

In [81]:
dfResMLDraw["ptResolHPT"]

0        0.000017
1        0.000624
2        0.000018
3        0.000083
4        0.000133
           ...   
11127    0.000188
11128    0.000020
11129    0.000111
11130    0.000256
11131    0.000003
Name: ptResolHPT, Length: 11132, dtype: float64

In [83]:
dfResMLDraw

,sigmaPtRel0,qPt,sigmaY0,sigmaZ0,sigmaqPt0,sigmaY0Rot,sigmaZ0Rot,sigmaqPt0Rot,pidCode,isSecondary,...,RF_10MeanMean,RF_10MeanMedian,RF_10MedianMedian,RF_10MeanRMS,RF_10MedianRMS,RF_10RMSMean,RF_10RMSMedian,FitRatio,ptResolHPT,FitRatioHPT
0,0.008931,0.069226,0.004346,0.004220,0.000618,0.004347,0.004220,0.000619,4,False,...,0.009490,0.009233,0.009335,0.001575,0.001562,0.000376,0.000454,0.963249,0.000017,0.001952
1,0.193233,0.172002,0.053499,0.035821,0.033236,0.053529,0.035865,0.033334,3,False,...,0.174592,0.190075,0.190611,0.042526,0.041583,0.008786,0.002885,1.099430,0.000624,0.003228
2,0.135303,6.778660,0.065963,0.061431,0.917171,0.060062,0.055461,0.535974,7,False,...,0.105789,0.109548,0.110289,0.032891,0.033330,0.011071,0.014436,1.131218,0.000018,0.000134
3,0.027310,0.046185,0.011385,0.007934,0.001261,0.011386,0.007934,0.001262,5,False,...,0.026724,0.024753,0.024518,0.005443,0.005379,0.001736,0.000748,1.044794,0.000083,0.003033
4,0.043622,0.033528,0.015079,0.010615,0.001463,0.015079,0.010616,0.001463,5,False,...,0.046889,0.046219,0.046167,0.004485,0.004327,0.001265,0.001373,0.956428,0.000133,0.003054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127,0.062588,0.021611,0.013731,0.010093,0.001353,0.013731,0.010094,0.001353,2,False,...,0.062864,0.063091,0.063063,0.003898,0.004292,0.001185,0.000381,0.996383,0.000188,0.003007
11128,0.014523,0.106843,0.001103,0.001128,0.001552,0.001103,0.001129,0.001553,3,False,...,0.033113,0.015700,0.014523,0.033606,0.030185,0.010549,0.002020,0.414031,0.000020,0.001411
11129,0.037560,0.122467,0.022779,0.016135,0.004600,0.022784,0.016144,0.004608,3,False,...,0.041198,0.042992,0.042774,0.015459,0.014899,0.004573,0.004786,0.966450,0.000111,0.002951
11130,0.083148,0.027831,0.024163,0.016249,0.002314,0.024166,0.016250,0.002315,2,False,...,0.086259,0.085679,0.085588,0.003179,0.003104,0.000813,0.000631,0.971613,0.000256,0.003079
